In [1]:
import numpy as np
from scipy.sparse.linalg import svds
#from alternating import optimize
from itertools import count
from scipy import sparse


In [2]:
datasetPath = '/media/yuanze/0C97197C0C97197C/files/WS16-17/mining_massive_data/project/train_triplets.txt'
n =300000
b = 10
ignoreLessThan = 5

In [3]:

row = []
col = []
data = []

user_to_row = {}
song_to_col = {}
user_count = count()
song_count = count()
limitingCounter = 0
with open(datasetPath) as f:
    for triplet in f:
        if limitingCounter ==n:
            break
        limitingCounter+=1
        
        uid, sid, play_count =triplet.split('\t')
        #print (uid, sid, play_count)
        if not uid in user_to_row:
            user_to_row[uid] = next(user_count)
        row.append(user_to_row[uid])
        if not sid in song_to_col:
            song_to_col[sid] = next(song_count)
        col.append(song_to_col[sid])        
        data.append(int(play_count))

$Preprocess 1:$

assign compress all entries in M into [i,i+1]

In [4]:
#for i in range(b-1):
    #new_items = [x if x % 2 else None for x in items]
#    data = [i+1 if x<=(2^(i+1)-1)and x>= 2^i else b for  x in data ]

#M = sparse.csr_matrix((data, (row, col)))
#print (M.shape)
## incorrect

In [5]:
data = np.floor(np.log(data)/np.log(2)+1)
data[data>b]=b
M = sparse.csr_matrix((data, (row, col)))
print(M.shape)

(6270, 83822)


$Preprocess 2:$

Remove invalid songs and users

In [6]:
size = M.shape
count = 0
while(True):
    count += 1
    mask = M!=0
    dr = mask.sum(axis=1)
    M = M[np.where(dr>ignoreLessThan)[0],:]
    mask = M!=0
    dc = mask.sum(axis=0)
    M = M[:,np.where(dc>ignoreLessThan)[1]]
    if size == M.shape:
        break
    size = M.shape
print(M.shape, count)
# the result is wired

((5693, 10966), 5)
